In [ ]:
#!/usr/bin/env python
# coding: utf-8
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '../')
from pyMulticopterSim.simulation.env import *

# execute only if run as a script
env = simulation_env()
env.proceed_motor_speed("uav1", np.array([1100.0,1100.0,1100.0,1100.0]),0.1)
env.plot_state("uav1")

In [ ]:
#!/usr/bin/env python
# coding: utf-8
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os, sys, time, copy, yaml
from scipy.special import factorial, comb, perm
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
import h5py

sys.path.insert(0, '../')
from pyTrajectoryUtils.pyTrajectoryUtils.utils import *
from multicopter_dynamics_sim import MulticopterDynamicsSim as uav_dyns
from pyMulticopterSim.simulation.env import *
from pyTrajectoryUtils.pyTrajectoryUtils.PIDcontroller import *

def plot_state(time_array, state, state_ref, label_txt='vel', dim=3, flag_save=False):
    start_idx = 0
    if failure_idx >= 0:
        end_idx = min(status_ref.shape[0], state.shape[0], time_array.shape[0], failure_idx)
    else:
        end_idx = min(status_ref.shape[0], state.shape[0], time_array.shape[0])
    time_array_t = time_array[start_idx:end_idx]

    plt.ioff()
    fig = plt.figure(figsize=(10,5))
    ax = fig.add_subplot(111)
    for i in range(dim):
        ax.plot(time_array_t, state[start_idx:end_idx,i], '-', label='{} dim {}'.format(label_txt,i))
        ax.plot(time_array_t, state_ref[start_idx:end_idx,i], '-', label='{} ref dim {}'.format(label_txt,i))
        ax.legend()
    ax.grid()
    plt.show()
    plt.pause(0.1)
    if flag_save:
        plt.savefig('{}/{}_{}.png'.format(save_dir,save_idx,label_txt))
        plt.close()
        
if __name__ == "__main__":
    env = simulation_env()
    controller = UAV_pid_tracking()
    
    traj_ref_path = '../test/sample_trajectory.csv'
    df = pd.read_csv(traj_ref_path, sep=',', header=None)
    status_ref = df.values[1:,:]
    print(status_ref.shape)
    
    freq_ctrl=200
    freq_sim=2000
    max_time = 100
    
    dt_micro_ctrl = np.int(1e6/freq_ctrl)
    freq_sim_update = np.int(freq_sim/freq_ctrl)
    N = min(status_ref.shape[0], max_time*freq_ctrl)
            
    traj_ref = status_ref[0,:]
    curr_time = 0

    env.set_state_vehicle("uav1", position=status_ref[0,2:5], velocity=status_ref[0,5:8])
    state_t = env.get_state("uav1")

    pos = state_t["position"]
    vel = state_t["velocity"]
    acc = state_t["acceleration"]
    att_q = state_t["attitude"]
    att = state_t["attitude_euler_angle"]

    angV = state_t["angular_velocity"]
    angA = state_t["angular_acceleration"]
    ms = state_t["motor_speed"]
    ma = state_t["motor_acceleration"]
    
    raw_acc = state_t["acceleration_raw"]
    raw_gyro = state_t["gyroscope_raw"]
    raw_ms = state_t["motor_speed_raw"]

    pos_array = np.zeros((N,3))
    vel_array = np.zeros((N,3))
    acc_array = np.zeros((N,3))
    att_array = np.zeros((N,3))
    att_q_array = np.zeros((N,4))
    raw_acc_array = np.zeros((N,3))
    raw_gyro_array = np.zeros((N,3))
    filtered_acc_array = np.zeros((N,3))
    filtered_gyro_array = np.zeros((N,3))
    ms_array = np.zeros((N,4))
    ms_c_array = np.zeros((N,4))
    time_array = np.zeros(N)
    pos_err_array = np.zeros(N)
    yaw_err_array = np.zeros(N)
    failure_idx = -1
    failure_start_idx = -1
    failure_end_idx = -1

    for it in range(N):
        curr_time = np.int(1.0*(it+1)/freq_ctrl*1e6)

        traj_ref = status_ref[it,2:]
        pos_ref = traj_ref[:3]
        vel_ref = traj_ref[3:6]
        
        ms_c = controller.control_update(traj_ref, pos, vel, acc, att, angV, angA, 1.0/freq_ctrl)
        env.proceed_motor_speed("uav1", ms_c, 1.0/freq_ctrl)
        
        state_t = env.get_state("uav1")

        pos = state_t["position"]
        vel = state_t["velocity"]
        acc = state_t["acceleration"]
        att_q = state_t["attitude"]
        att = state_t["attitude_euler_angle"]
        angV = state_t["angular_velocity"]
        angA = state_t["angular_acceleration"]
        ms = state_t["motor_speed"]
        ma = state_t["motor_acceleration"]
        
        raw_acc = state_t["acceleration_raw"]
        raw_gyro = state_t["gyroscope_raw"]
        raw_ms = state_t["motor_speed"]

        time_array[it] = 1.0*(it+1)/freq_ctrl
        pos_array[it,:] = pos
        vel_array[it,:] = vel
        acc_array[it,:] = acc
        att_array[it,:] = att
        att_q_array[it,:] = att_q
        raw_acc_array[it,:] = raw_acc
        raw_gyro_array[it,:] = raw_gyro
        filtered_acc_array[it,:] = acc
        filtered_gyro_array[it,:] = angV
        ms_array[it,:] = ms
        ms_c_array[it,:] = ms_c

    plot_state(time_array, pos_array, status_ref[:,2:5], label_txt='pos', dim=3)


In [ ]:
#!/usr/bin/env python
# coding: utf-8
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os, sys, time, copy, yaml
sys.path.insert(0, '../')
from pyTrajectoryUtils.pyTrajectoryUtils.trajectorySimulation import *

traj_sim = TrajectorySimulation()
res = traj_sim.run_simulation(traj_ref_path='./sample_trajectory.csv', N_trial=1, 
                       max_pos_err=5.0, min_pos_err=0.5, 
                       max_yaw_err=30.0, min_yaw_err=5.0, 
                       freq_ctrl=200)
traj_sim.plot_result(debug_value=res[0], flag_save=False)